In [1]:
pip install requests
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [15]:
import requests
import pandas as pd

API_KEY = 'F-MMiRXPM4HmVbiYChYiwz-qNuVwELkbueD66N4M8x8iFQDiZ8CQGarwJ1trERkHPiNCTKhCZcY1B6aiSw6WWAmE_Hr4AEDB2e4ilZpXlC0lptX4IBRhnz-OF44qaHYx'
HEADERS = {'Authorization': f'Bearer {API_KEY}'}

def get_yelp_cat_cafes(location, limit=50):
    url = 'https://api.yelp.com/v3/businesses/search'
    params = {
        'term': 'cat cafe',
        'location': location,
        'categories': 'cafes',
        'limit': limit
    }
    response = requests.get(url, headers=HEADERS, params=params)
    data = response.json()
    
    cafes = []
    for biz in data.get('businesses', []):
        cafes.append({
            'Name': biz['name'],
            'Rating': biz['rating'],
            'Review Count': biz['review_count'],
            'Price': biz.get('price'),
            'Phone': biz['phone'],
            'Address': ", ".join(biz['location']['display_address']),
            'Categories': ", ".join([cat['title'] for cat in biz['categories']]),
            'Latitude': biz['coordinates']['latitude'],
            'Longitude': biz['coordinates']['longitude'],
            'Yelp URL': biz['url'],
            'Is Closed': biz['is_closed']
        })

    return pd.DataFrame(cafes)


In [17]:
# Example usage:
df = get_yelp_cat_cafes("San Francisco, CA")
print(df.head())
df.to_csv("cat_cafes_sf_yelp_api.csv", index=False)

                    Name  Rating  Review Count Price         Phone  \
0                 KitTea     4.6          1078    $$  +14153255510   
1          Nabi Cat Cafe     4.8            67  None                 
2        Jade Chocolates     4.4            82  None  +14153503878   
3  Little Beans Cat Cafe     0.0             0  None  +14083419061   

                                     Address  \
0  1266 Valencia St, San Francisco, CA 94110   
1    2255 The Alameda, Santa Clara, CA 95050   
2     607 Grant Ave, San Francisco, CA 94108   
3   15365 Monterey Rd, Morgan Hill, CA 95037   

                                          Categories   Latitude   Longitude  \
0                         Pet Adoption, Themed Cafes  37.752640 -122.420933   
1           Themed Cafes, Pet Adoption, Coffee & Tea  37.344552 -121.933203   
2                 Themed Cafes, Chocolatiers & Shops  37.792660 -122.406120   
3  Themed Cafes, Pet Adoption, Community Service/...  37.106647 -121.634304   

            

In [21]:
SEARCH_URL = "https://api.yelp.com/v3/businesses/search"

HEADERS = {
    "Authorization": f"Bearer {API_KEY}"       #This sets the HTTP headers required to authenticate the request to the Yelp Fusion API


}

    #"Authorization": This is the name of the HTTP header that tells Yelp, "Hey, I'm allowed to access your data!"

    # f"Bearer {API_KEY}": This is a formatted string that includes your personal Yelp API key.

    # The "Bearer " part is a standard way to present an API token.

    # {API_KEY} gets replaced with the actual key you got from Yelp’s developer site.



cities = [
    "San Francisco, CA", "Los Angeles, CA", "New York, NY",
    "Seattle, WA", "Austin, TX", "Chicago, IL", "Portland, OR",
    "Boston, MA", "Denver, CO", "Atlanta, GA"
]

def fetch_pet_cafes(city, term="pet cafe", limit=50):
    params = {
        "term": term,
        "location": city,
        "categories": "cafes",
        "limit": limit
    }

    try:
        response = requests.get(SEARCH_URL, headers=HEADERS, params=params)
        if response.status_code == 200:
            return response.json().get("businesses", [])
        else:
            print(f"Failed to fetch for {city}: {response.status_code}")
            return []
    except Exception as e:
        print(f"Error fetching {city}: {e}")
        return []

all_data = []

for city in cities:
    cafes = fetch_pet_cafes(city)
    print(f"Fetched {len(cafes)} results for {city}")
    for cafe in cafes:
        all_data.append({
            "name": cafe.get("name"),
            "city": city,
            "rating": cafe.get("rating"),
            "review_count": cafe.get("review_count"),
            "price": cafe.get("price", "N/A"),
            "address": ", ".join(cafe.get("location", {}).get("display_address", [])),
            "latitude": cafe.get("coordinates", {}).get("latitude"),
            "longitude": cafe.get("coordinates", {}).get("longitude"),
            "categories": ", ".join([cat["title"] for cat in cafe.get("categories", [])]),
            "is_closed": cafe.get("is_closed"),
            "url": cafe.get("url")
        })
    time.sleep(1)  # Avoid hitting Yelp rate limits

Fetched 29 results for San Francisco, CA
Fetched 50 results for Los Angeles, CA
Fetched 35 results for New York, NY
Fetched 11 results for Seattle, WA
Fetched 7 results for Austin, TX
Fetched 10 results for Chicago, IL
Fetched 6 results for Portland, OR
Fetched 14 results for Boston, MA
Fetched 7 results for Denver, CO
Fetched 3 results for Atlanta, GA


In [23]:
# Save to CSV
df = pd.DataFrame(all_data)
df.to_csv("pet_cafes.csv", index=False)
print("Saved to pet_cafes.csv")

Saved to pet_cafes.csv


In [25]:
CafeInfo = pd.read_csv(r"C:\Users\eshab\Downloads\PetCafeSuccess_Capstone2\pet_cafes.csv")
CafeInfo.head()

,name,city,rating,review_count,price,address,latitude,longitude,categories,is_closed,url
0,KitTea,"San Francisco, CA",4.6,1078,$$,"1266 Valencia St, San Francisco, CA 94110",37.752640,-122.420933,"Pet Adoption, Themed Cafes",False,https://www.yelp.com/biz/kittea-san-francisco-...
1,Xochi The Dog Cafe,"San Francisco, CA",4.6,110,NaN,"1038 E 21st St, Oakland, CA 94606",37.796204,-122.242281,"Cafes, Sandwiches",False,https://www.yelp.com/biz/xochi-the-dog-cafe-oa...
2,Spruce Cafe,"San Francisco, CA",4.0,622,$,"230 S Spruce Ave, South San Francisco, CA 94080",37.647570,-122.420350,"Cafes, Patisserie/Cake Shop, Sandwiches",False,https://www.yelp.com/biz/spruce-cafe-south-san...
3,Stable Cafe,"San Francisco, CA",4.0,697,$$,"2128 Folsom St, San Francisco, CA 94110",37.763238,-122.415311,"Breakfast & Brunch, Cafes",False,https://www.yelp.com/biz/stable-cafe-san-franc...
4,Moonlight Cafe,"San Francisco, CA",3.9,309,$$,"634 Cortland Ave, San Francisco, CA 94110",37.738876,-122.415282,"Cafes, Breakfast & Brunch, Sandwiches",False,https://www.yelp.com/biz/moonlight-cafe-san-fr...


In [ ]:
BELOW CODE IS NOT VIABLE YET since I do not have api key for foot traffic data yet. But the project is still possible with the above cleaned yelp data
    in case my project is not approved to receive an api key.

In [ ]:
FOOT_TRAFFIC_API_KEY = "your_foot_traffic_api_key_here"
HEADERS_FOOT = {
    "Authorization": f"Bearer {FOOT_TRAFFIC_API_KEY}"
}

import pandas as pd
from geopy.distance import geodesic
from fuzzywuzzy import fuzz


In [ ]:
# SafeGraph POI and Patterns data
poi_df = pd.read_csv("core_poi.csv")
patterns_df = pd.read_csv("patterns.csv")

#Filter SafeGraph for relevant business types ---

# Filter POIs by NAICS code for cafes (722513 = Limited-Service Restaurants)
poi_df = poi_df[poi_df["naics_code"] == 722513]

# Drop rows without coordinates
poi_df = poi_df.dropna(subset=["latitude", "longitude"])

# Match POIs with Yelp cat cafes based on name and location ---

def match_poi(cafe_row, poi_df, distance_threshold_km=0.5, name_similarity_threshold=70):
    cafe_coords = (cafe_row["latitude"], cafe_row["longitude"])
    matches = []
    
    for _, cafe_row in poi_df.iterrows():
        poi_coords = (poi_row["latitude"], poi_row["longitude"])
        distance = geodesic(cafe_coords, poi_coords).km
        
        name_score = fuzz.partial_ratio(str(yelp_row["name"]).lower(), str(poi_row["location_name"]).lower())
        
        if distance < distance_threshold_km and name_score > name_similarity_threshold:
            matches.append({
                "cafe_name": cafe_row["name"],
                "safegraph_name": poi_row["location_name"],
                "poi_cbg": poi_row["location_cbg"],
                "safegraph_place_id": poi_row["safegraph_place_id"],
                "distance_km": distance,
                "name_score": name_score
            })
    
    return pd.DataFrame(matches)

# Run match for each Yelp café and collect results
match_results = pd.concat([match_poi(row, poi_df) for _, row in CafeInfo_df.iterrows()], ignore_index=True)

# --- STEP 4: Merge Yelp data with matched SafeGraph place IDs ---

# Only take best match per Yelp café (if multiple)
match_results = match_results.sort_values(by=["yelp_name", "name_score", "distance_km"], ascending=[True, False, True])
best_matches = match_results.groupby("yelp_name").head(1)

# Merge with Yelp data
yelp_with_ids = pd.merge(yelp_df, best_matches, left_on="name", right_on="yelp_name", how="left")

In [ ]:
# Merge with SafeGraph visit patterns
full_data = pd.merge(yelp_with_ids, patterns_df, on="safegraph_place_id", how="left")

In [ ]:
# Analysis: Traffic vs Yelp Reviews ---

import matplotlib.pyplot as plt
import seaborn as sns

# Convert visits_by_day (stringified list) to weekly average visits
def parse_visits(visits_str):
    try:
        visits = eval(visits_str)
        return sum(visits) / len(visits) if visits else 0
    except:
        return 0

full_data["avg_daily_visits"] = full_data["visits_by_day"].apply(parse_visits)

In [ ]:
# Plot: Reviews vs Foot Traffic
plt.figure(figsize=(10, 6))
sns.scatterplot(x="review_count", y="avg_daily_visits", hue="city", data=full_data)
plt.title("Yelp Review Count vs. Foot Traffic (SafeGraph)")
plt.xlabel("Yelp Review Count")
plt.ylabel("Average Daily Visits")
plt.grid(True)
plt.tight_layout()
plt.show()

# Optional: Save merged file
full_data.to_csv("cat_cafes_with_foot_traffic.csv", index=False)